# Projeto 1 - Ciência dos Dados

Nome: Lourenço Meirelles

Nome: Lucca Lima

Nome: Pedro Drumond

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [16]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [17]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Pedro Drumond\Documents\Insper 2o semestre\CDados\GitHub_CD\Projeto_1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [18]:
filename = 'Primevideo.xlsx'

In [19]:
train = pd.read_excel(filename)
test = pd.read_excel(filename, sheet_name = 'Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O Amazon Prime Video é uma plataforma de streaming semelhante à Netflix, na qual você pode acessar um catálogo de filmes, séries, etc... além de disponibilizar outras coisas como o Amazon Prime Music e entre outros.

Para classificar, consideramos mensagens "automáticas", prontas, divulgando links como 'muito irrelevantes'. Mensagens aleatórias e não relacionadas ao assunto como 'irrelevantes'. Mensagens que elogiassem a plataforma como 'relevantes' e mensagens que elogiassem muito a plataforma como 'muito relevantes'.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [20]:
def cleanup(text):
    #import string
    punctuation = '[\n!-.:@?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    #MELHORAR
    
#tirar '@'
#talvez 'e', 'que'

In [21]:
#DAR UMA OLHADA EM PANDAS
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
primevideo = pd.read_excel('Primevideo.xlsx')
primevideo.Classificacao = primevideo.Classificacao.astype('int')
primevideo_teste = pd.read_excel('Primevideo.xlsx', sheet_name=1)

In [22]:
#Criar um DF (DataFrame) para cada classificação

#Filtra as linhas, separando cada clasificação em um DF sem ter aplicado o 'CleanUp'
filtra_linhas = primevideo.Classificacao == 0
primevideo_classfic_0_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

filtra_linhas = primevideo.Classificacao == 1
primevideo_classfic_1_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 2
primevideo_classfic_2_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 3
primevideo_classfic_3_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

print(primevideo_classfic_0_raw)

0      @artofesthetic baseado no caso ritchofen, dois...
1      @laressa baseado no caso ritchofen, dois filme...
2      @popzonebr baseado no caso ritchofen, dois fil...
3      @bbb @gshow baseado no caso ritchofen, dois fi...
4      @flamengo @davidluiz_4 baseado no caso ritchof...
                             ...                        
142    @paulaamorim baseado no caso ritchofen, dois f...
143    @eulaurakeller @valescaoficial baseado no caso...
144    @sepcarloslima @acordadebora um crime real, do...
145    @acordadebora um crime real, dois filmes e dua...
146    @rolealeatorio baseado no caso ritchofen, dois...
Name: Treinamento, Length: 147, dtype: object


In [23]:
#For, criando as listas limpas
lista_primevideo_classfic_0 = []
lista_primevideo_classfic_1 = []
lista_primevideo_classfic_2 = []
lista_primevideo_classfic_3 = []

lista_com_todos_tuites_limpos = []

#Cria a lista lista_primevideo_classfic que possui  as classificações de cada categoria
lista_tuites = []
for celula in primevideo_classfic_0_raw:
    
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_0 += celula_limpa.split()
    #print(lista_primevideo_classfic_0)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_1_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_1 += celula_limpa.split()
    #print(lista_primevideo_classfic_1)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_2_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_2 += celula_limpa.split()
    #print(lista_primevideo_classfic_2)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_3 += celula_limpa.split()
    #print(lista_primevideo_classfic_3)
    #lista_tuites.append(celula_limpa.split())
    
'''
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower()).split()
    lista_primevideo_classfic_3 += celula_limpa
    print(lista_primevideo_classfic_3)
'''



'\nfor celula in primevideo_classfic_3_raw:\n    celula_limpa = cleanup(celula.lower()).split()\n    lista_primevideo_classfic_3 += celula_limpa\n    print(lista_primevideo_classfic_3)\n'

In [24]:
# cria a lista tuite a partir da planilha Teste
for tuite in primevideo_teste['Teste']:
    celula_limpa = cleanup(tuite.lower())
    lista_tuites.append(celula_limpa.split())

In [25]:
#Convertendo as tabelas para pd.Series

classfic_0 = pd.Series(lista_primevideo_classfic_0)
classfic_1 = pd.Series(lista_primevideo_classfic_1)
classfic_2 = pd.Series(lista_primevideo_classfic_2)
classfic_3 = pd.Series(lista_primevideo_classfic_3)

In [26]:
#Tabelas de contagem de ocorrencias em cada tabela, usando o value_counts()
tabela_0 = classfic_0.value_counts()
tabela_1 = classfic_1.value_counts()
tabela_2 = classfic_2.value_counts()
tabela_3 = classfic_3.value_counts()

#Juntando todos os tuítes
todos_tuites = lista_primevideo_classfic_0 + lista_primevideo_classfic_1 + lista_primevideo_classfic_2 + lista_primevideo_classfic_3
#Criando tabela dos tuítes usando o pd.Series
todos_tuites_lista = pd.Series(todos_tuites)
tabela_todos_tuites = todos_tuites_lista.value_counts()
tabela_todos_tuites
#tabela_classific_0

#ACHO QUE VAMOS TER QUE USAR TABELA RELATIVA PORQUE TEMOS MAIS AVALIAÇÕES EM UNS DO QUE OUTROS, VER ISSO!

e                        248
prime                    225
primevideo               208
a                        207
que                      198
                        ... 
ansiosamente               1
convivem                   1
https//tco/qoiidkvfmj      1
multishow                  1
high                       1
Length: 2552, dtype: int64

In [27]:
prob_ser_0 = len(classfic_0)/len(tabela_todos_tuites)
prob_ser_1 = len(classfic_1)/len(tabela_todos_tuites)
prob_ser_2 = len(classfic_2)/len(tabela_todos_tuites)
prob_ser_3 = len(classfic_3)/len(tabela_todos_tuites)

In [28]:
classificacao_final = []
for tuite in lista_tuites:
    prob_0 = 1
    prob_1 = 1
    prob_2 = 1
    prob_3 = 1
    
    #dentro desse FOR teremos: -probabilidade das palavras, -descobre todas as probabilidades, -aplica La Place, -descobre qual a classificacao
    for palavra in tuite:
        
        #LA PLACE
        if palavra not in tabela_0:
            prob_0 *= 1/(len(classfic_0) + len(tabela_todos_tuites)) 
            #/todas as palavras na categoria + todas as palavras de todas as categorias (sem repetição)
            #acho que não está pegando todas as palavras ta tabela 0           
        elif palavra in tabela_0:
            prob_0 *= (tabela_0[palavra] + 1)/(len(classfic_0) + len(tabela_todos_tuites))
        
        if palavra not in tabela_1:
            prob_1 *= 1/(len(classfic_1) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
                          #quantida de palavras na categoria 1 + quantidade de palavras no total
        elif palavra in tabela_1:
            prob_1 *= (tabela_1[palavra] + 1)/(len(classfic_1) + len(tabela_todos_tuites))
        
        if palavra not in tabela_2:
            prob_2 *= 1/(len(classfic_2) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
        elif palavra in tabela_2:
            prob_2 *= (tabela_2[palavra] + 1)/(len(classfic_2) + len(tabela_todos_tuites))
        
        if palavra not in tabela_3:
            prob_3 *= 1/(len(classfic_3) + len(tabela_todos_tuites)) #talvez seja a tabela errada, rever porque isso é do 'Treinamento'
        elif palavra in tabela_3:
            prob_3 *= (tabela_3[palavra] + 1)/(len(classfic_3) + len(tabela_todos_tuites))
        
    #cálculo das probabilidades
    prob_tuite_dado_0 = prob_0 * prob_ser_0
    prob_tuite_dado_1 = prob_1 * prob_ser_1
    prob_tuite_dado_2 = prob_2 * prob_ser_2
    prob_tuite_dado_3 = prob_3 * prob_ser_3

    #COMPARAÇÃO ENTRE OS RESULTADOS DE LA PLACE PARA CADA CATEGORIA
    if prob_tuite_dado_0 > prob_tuite_dado_1 and prob_tuite_dado_0 > prob_tuite_dado_2 and prob_tuite_dado_0 > prob_tuite_dado_3:
        #print(prob_0)
        classificacao_final.append(0)
        #adicionar na coluna no excel
            
    elif prob_tuite_dado_1 > prob_tuite_dado_0 and prob_tuite_dado_1 > prob_tuite_dado_2 and prob_tuite_dado_1 > prob_tuite_dado_3:
        #print(prob_1)
        classificacao_final.append(1)
            
    elif prob_tuite_dado_2 > prob_tuite_dado_0 and prob_tuite_dado_2 > prob_tuite_dado_1 and prob_tuite_dado_2 > prob_tuite_dado_3:
        #print(prob_2)
        classificacao_final.append(2)
            
    elif prob_tuite_dado_3 > prob_tuite_dado_0 and prob_tuite_dado_3 > prob_tuite_dado_1 and prob_tuite_dado_3 > prob_tuite_dado_2:
        #print(prob_3)
        classificacao_final.append(3)
        
        # denominador: ocorrencia da palavra na categoria + 1 / todas as palavras na categoria + todas as palavras de todas as categorias (sem repetição)
        # prob_A_dado_B = prob_B_dado_A * prob_ser_0
    
        
        
        #Chama função que faz o calculo de P_classificação_dado_tuite:
        
        #prob_0_dado_tuite = prob_0 * ( len(tabela_0) / len(tabela_todos_tuites )
        #                                                        esse len() está errado
        
        '''
        prob_0_dado_tuite = calcula_prob_classific_dado_tuite(prob_0, len(tabela_0), len(tabela_todos_tuites))
        prob_1_dado_tuite = calcula_prob_classific_dado_tuite(prob_1, len(tabela_1), len(tabela_todos_tuites))
        prob_2_dado_tuite = calcula_prob_classific_dado_tuite(prob_2, len(tabela_2), len(tabela_todos_tuites))
        prob_3_dado_tuite = calcula_prob_classific_dado_tuite(prob_3, len(tabela_3), len(tabela_todos_tuites))
        '''


#FAZER ALGUMA COISA COM O 'classificacao_final' QUE É UMA TABELA COM AS CLASSIFICAÇÕES

In [31]:
#VERIFICANDO A PERFORMANCE

classificacao_final
classificacao_final_pd = pd.Series(classificacao_final)
primevideo_teste['Resultado Classificador'] = classificacao_final_pd
primevideo_teste.tail(20)

pd.crosstab(primevideo_teste['Classificação'], primevideo_teste['Resultado Classificador'], margins=True)#linha e coluna

Resultado Classificador,0,1,2,3,All
Classificação,,,,,
0,49,20,5,7,81
1,7,94,20,8,129
2,1,13,35,3,52
3,1,6,18,12,37
All,58,133,78,30,299


In [54]:
pd.crosstab(primevideo_teste['Classificação'], primevideo_teste['Resultado Classificador'], normalize=True).round(4)*100#linha e coluna

Resultado Classificador,0,1,2,3
Classificação,,,,
0,1.00,4.68,16.72,4.68
1,1.34,8.36,25.75,7.69
2,0.00,1.00,8.36,8.03
3,0.33,0.33,6.02,5.69


*ANÁLISE CRÍTICA:*

Nosso classificador conseguiu distinguir bem as classificações entre 0 e 1, provavelmente porque 
possuem a maior frequência dentre todas as classificações, assim ele "aprende" melhor as diferenças 
dessas duas classificações.
Apesar de ele ter acertado algumas entre 2 e 3, sua precisão foi inferior comparado as duas primeiras
classificações, provavelmente devido à menor quantidade de tweets dentro dessas classificações, dificultando
o "aprendizado" do classificador. Porém, mesmo com esse déficit, a maior quantidade foi de acertos.

O nosso classificador não é capaz de identificar a diferença das mensagens de dupla negação e sarcasmo.
Pois ele avalia as pelavras individualmente, as tirando do contexto da frase (tweet), considerando apenas
suas ocorrências.

Vocês devem continuar ou melhorar o financiando do nosso projeto porque com o aumento investimento seria
possível coletar mais tweets, principalmente de classificação 2 e 3, melhorando o "aprendizado" do 
classificador e possivelmente criar mais classificações, o deixando mais preciso.





*POR QUE NÃO PODEMOS USAR NOSSO CLASSIFICADOR PARA GERAR MAIS AMOSTRAS DE TREINAMENTO?*

Não é possível utilizar o classificador para gerar mais amostras de treinamento porque
a interpretação dele é diferente da nossa, ele é capaz de aprender as probabilidades
das palavras de nossas interpretações, não interpretar.

Além disso, ainda há certas coisas que o classificador está categorizando erroneamente, seria possível corrigí-las aumentando a quantidade de tweets em seu aprendizado.

melhorar - *DIFERENTES CENÁRIOS PARA NAIVE BAYES FORA DO CONTEXTO DO PROJETO:*
- Fazer previsões em tempo real
- Prever se um usuário gostaria de um determinado recurso ou não

melhorar - *MELHORIAS REAIS:*
- Melhorar os critérios das classificações do aprendizado
- Aumentar o número de classificações
- Aumentar a quantidade de dados coletados (tweets)

In [1]:
'''
CONCEITO A:
- Melhorar o cleanup()
- Corrigir espaçamento nos tuítes
- 
'''

'\nCONCEITO A:\n- Melhorar o cleanup()\n- Corrigir espaçamento nos tuítes\n- Explicar porque não podemos usar nosso classificador para gerar mais amostras de treinamento\n- Difirentes cenários para Naive Bayes fora do contexto do projeto\n- Sugerir melhorias reais \n- \n'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**